In [1]:
import pandas as pd

In [121]:
df = pd.read_csv('KC_case_data.csv', parse_dates = ['date'])

In [122]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [123]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [124]:
df.shape

(2747968, 8)

In [125]:
df_february = df.query('date.dt.month == 2')

In [126]:
df_february.device_id.nunique()

75032

In [127]:
df.query('date.dt.month == 1').event.value_counts()

app_start      268471
search         254732
choose_item    196137
tap_basket     140616
app_install     80297
purchase        59208
register        42946
Name: event, dtype: int64

In [128]:
df_conv = pd.DataFrame(index=df.date.unique(), columns=['a'])

In [129]:
df_conv['users_with_install'] = df.query('event == "app_install"').groupby('date').agg({'device_id':'nunique'})

In [130]:
df_conv

,a,users_with_install
2020-01-01,NaN,3579
2020-01-02,NaN,3144
2020-01-03,NaN,2402
2020-01-04,NaN,1831
2020-01-05,NaN,1671
...,...,...
2020-03-27,NaN,1199
2020-03-28,NaN,1091
2020-03-29,NaN,1117
2020-03-30,NaN,994


In [133]:
first_purc = df.query('event == "purchase"')\
               .groupby('device_id')\
               .agg({'date':'min'})\
               .reset_index()

In [134]:
first_purc

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [147]:
from datetime import timedelta

In [229]:
df_merged = df.query('event == "app_install"').merge(first_purc, how='inner', on='device_id')

In [230]:
df_merged

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_y
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10
1,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01
2,2020-01-01,app_install,NaN,android,8423608,female,Saint-Petersburg,referal,2020-01-06
3,2020-01-01,app_install,NaN,android,532710,male,Moscow,yandex-direct,2020-01-03
4,2020-01-01,app_install,NaN,android,528071,female,Saint-Petersburg,vk_ads,2020-01-02
...,...,...,...,...,...,...,...,...,...
57453,2020-03-31,app_install,NaN,ios,228794,female,Saint-Petersburg,facebook_ads,2020-03-31
57454,2020-03-31,app_install,NaN,android,20224705,female,Moscow,instagram_ads,2020-03-31
57455,2020-03-31,app_install,NaN,android,1434534,male,Moscow,referal,2020-03-31
57456,2020-03-31,app_install,NaN,android,271586,male,Moscow,-,2020-03-31


In [231]:
df_merged['diff_time'] = df_merged.date_y - df_merged.date_x

In [232]:
df_merged.dtypes

date_x           datetime64[ns]
event                    object
purchase_sum            float64
os_name                  object
device_id                 int64
gender                   object
city                     object
utm_source               object
date_y           datetime64[ns]
diff_time       timedelta64[ns]
dtype: object

In [233]:
df_merged

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_y,diff_time
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,9 days
1,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,0 days
2,2020-01-01,app_install,NaN,android,8423608,female,Saint-Petersburg,referal,2020-01-06,5 days
3,2020-01-01,app_install,NaN,android,532710,male,Moscow,yandex-direct,2020-01-03,2 days
4,2020-01-01,app_install,NaN,android,528071,female,Saint-Petersburg,vk_ads,2020-01-02,1 days
...,...,...,...,...,...,...,...,...,...,...
57453,2020-03-31,app_install,NaN,ios,228794,female,Saint-Petersburg,facebook_ads,2020-03-31,0 days
57454,2020-03-31,app_install,NaN,android,20224705,female,Moscow,instagram_ads,2020-03-31,0 days
57455,2020-03-31,app_install,NaN,android,1434534,male,Moscow,referal,2020-03-31,0 days
57456,2020-03-31,app_install,NaN,android,271586,male,Moscow,-,2020-03-31,0 days


In [234]:
df_merged = df_merged[(df_merged.diff_time <= timedelta(days=7)) & (df_merged.diff_time >= timedelta(days=0))]

In [236]:
df_conv['with_purch_in_7D'] = df_merged.groupby('date_x').device_id.nunique()

In [237]:
df_conv['convertion'] = df_conv.with_purch_in_7D / df_conv.users_with_install


In [238]:
df_conv.convertion.idxmax()

Timestamp('2020-01-01 00:00:00')

In [239]:
df_merged.sort_values('date_x')

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_y,diff_time
1,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,0 days
1539,2020-01-01,app_install,NaN,android,33804036,male,Moscow,-,2020-01-01,0 days
1538,2020-01-01,app_install,NaN,ios,3265400,male,Saint-Petersburg,-,2020-01-01,0 days
1537,2020-01-01,app_install,NaN,ios,1882985,male,Moscow,google_ads,2020-01-01,0 days
1534,2020-01-01,app_install,NaN,ios,9885,male,Saint-Petersburg,facebook_ads,2020-01-08,7 days
...,...,...,...,...,...,...,...,...,...,...
57302,2020-03-31,app_install,NaN,ios,3135794,female,Saint-Petersburg,instagram_ads,2020-03-31,0 days
57303,2020-03-31,app_install,NaN,android,999691,male,Moscow,google_ads,2020-03-31,0 days
57304,2020-03-31,app_install,NaN,ios,26043121,female,Moscow,vk_ads,2020-03-31,0 days
57275,2020-03-31,app_install,NaN,ios,3458168,female,Saint-Petersburg,referal,2020-03-31,0 days


In [240]:
df_conv.sort_values('convertion', ascending=False)

,a,users_with_install,with_purch_in_7D,convertion
2020-01-01,NaN,3579,1408,0.393406
2020-01-09,NaN,1424,558,0.391854
2020-01-15,NaN,4310,1650,0.382831
2020-01-14,NaN,5173,1973,0.381403
2020-01-02,NaN,3144,1186,0.377226
...,...,...,...,...
2020-03-18,NaN,1171,235,0.200683
2020-03-29,NaN,1117,223,0.199642
2020-03-22,NaN,1261,251,0.199048
2020-03-28,NaN,1091,209,0.191567


In [243]:
df_merged.utm_source.value_counts()

-                9795
yandex-direct    8793
google_ads       7964
vk_ads           7169
instagram_ads    5925
facebook_ads     4226
referal          2802
Name: utm_source, dtype: int64

In [245]:
df.query('event == "search"').device_id.nunique()

184488

In [246]:
df.query('event == "choose_item"').device_id.nunique()

155691

In [247]:
df.query('event == "tap_basket"').device_id.nunique()

125414

In [248]:
df.query('event == "register"').device_id.nunique()

78310

In [259]:
(155691)/184488

0.8439085468973592

In [260]:
125414/155691

0.8055314693848713

In [261]:
78310/125414

0.6244119476294513

In [249]:
date_register = df.query('event == "register"')\
               .groupby('device_id')\
               .agg({'date':'min'})\
               .reset_index()

In [251]:
date_register

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4067,2020-01-31
4,4120,2020-01-07
...,...,...
78305,35342310,2020-03-27
78306,35358658,2020-03-27
78307,35367797,2020-03-29
78308,35370030,2020-03-31


In [252]:
df_with_register = df.merge(date_register, how='inner', on='device_id')

In [253]:
df_with_register

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_y
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,2020-01-07
...,...,...,...,...,...,...,...,...,...
2205961,2020-03-31,purchase,654.0,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31
2205962,2020-03-31,search,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31
2205963,2020-03-31,tap_basket,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31
2205964,2020-03-31,app_install,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31


In [254]:
df_with_register['diff_time'] = df_with_register.date_x - df_with_register.date_y

In [256]:
df_with_register[df_with_register.diff_time <= timedelta(days=0)]

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_y,diff_time
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07,-6 days
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,2020-01-07,-6 days
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,2020-01-07,-4 days
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,2020-01-07,-4 days
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,2020-01-07,-4 days
...,...,...,...,...,...,...,...,...,...,...
2205961,2020-03-31,purchase,654.0,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31,0 days
2205962,2020-03-31,search,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31,0 days
2205963,2020-03-31,tap_basket,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31,0 days
2205964,2020-03-31,app_install,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03-31,0 days


In [257]:
df_with_register.query('event == "register"').device_id.nunique()

78310

In [262]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [263]:
df_utm = df.query('event == "app_install"').groupby('utm_source').agg({'device_id':'nunique'})

In [265]:
df_utm.rename(columns={'device_id':'qua_install'},inplace=True)

In [266]:
df_utm['qua_purc'] = df.query('event == "purchase"').groupby('utm_source').agg({'device_id':'nunique'})

In [268]:
df_utm['cr'] = df_utm.qua_purc/df_utm.qua_install

In [270]:
df_utm.sort_values('cr', ascending=False)

,qua_install,qua_purc,cr
utm_source,,,
referal,9282,6362,0.685413
facebook_ads,13916,9017,0.647959
instagram_ads,20096,10762,0.535529
vk_ads,23189,12364,0.533184
-,32460,16598,0.511337
google_ads,26286,11339,0.431370
yandex-direct,29368,12028,0.409561


In [273]:
df_median_check = df.query('event == "purchase"').merge(first_purc, how='inner', on=['device_id', 'date'])

In [276]:
df_median_check.groupby('utm_source').agg({'purchase_sum':'median'}).sort_values('purchase_sum')

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


In [278]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [279]:
df_median_check.groupby('utm_source').agg({'purchase_sum':'sum'}).sort_values('purchase_sum')

,purchase_sum
utm_source,
referal,4041326.0
facebook_ads,5469587.5
instagram_ads,7033094.0
google_ads,7126255.0
yandex-direct,7936700.5
vk_ads,8210863.0
-,10323749.0
